## Градиентный спуск с постоянным шагом

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [3]:
from Functions.user_functions import f_well, f_poor, f_rozen, grad_well, grad_poor, grad_rozen
from Functions.optimization import classic_grad_descent

In [4]:
N_LR = 5
lr_classic_grad_descent = np.empty((N_LR,), dtype=object)
for n in range(N_LR):
    lr_classic_grad_descent[n] = np.array([1/10**(n+1+2), 1/10**(n+1+2)])
    
for lr in lr_classic_grad_descent:
    print(lr)

[0.001 0.001]
[0.0001 0.0001]
[1.e-05 1.e-05]
[1.e-06 1.e-06]
[1.e-07 1.e-07]


In [15]:
lr_classic_grad_descent = np.delete(lr_classic_grad_descent, [0,1])
for lr in lr_classic_grad_descent:
    print(lr)
N_LR = len(lr_classic_grad_descent)

[0.001 0.001]
[0.0001 0.0001]
[1.e-05 1.e-05]


In [14]:
N_EPS = 5
EPS_INIT = -1
eps_var = np.logspace(EPS_INIT, EPS_INIT-N_EPS+1, N_EPS)
print(eps_var)

[1.e-01 1.e-02 1.e-03 1.e-04 1.e-05]


In [23]:
func_names = [f_well]#, f_poor, f_rozen]
grad_names = [grad_well]#, grad_poor, grad_rozen]
func_labels = ['Well-conditioned']#,
               #'Poorly-conditioned',
               #'Rosenbrock']

N_FUNC = len(func_names)

In [24]:
x0_optim_arr = np.empty((N_FUNC, N_LR), dtype='object')
x1_optim_arr = np.empty((N_FUNC, N_LR), dtype='object')
#losses_arr = np.empty((N_FUNC, N_LR), dtype='object')
iter_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
func_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
grad_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')

# Начальное приближение.
x_init = np.array([2.2, 3.3])

for ndx, func, grad in zip(range(N_FUNC), func_names, grad_names):
    for mdx, lr in enumerate(lr_classic_grad_descent):
        x0_optim_list = []
        x1_optim_list = []
        iter_counter_list = []
        func_counter_list = []
        grad_counter_list = []
    
        for idx, eps in enumerate(eps_var):
            x_optim, _, iter_final, func_counter, grad_counter = classic_grad_descent(
                                                                                        loss_func=func,
                                                                                        grad_func=grad,
                                                                                        x_init=x_init,
                                                                                        learning_rate=lr,
                                                                                        tolerance=eps,
                                                                                        printoutput=False
                                                                                        )
            x0_optim_list.append(x_optim[0])
            x1_optim_list.append(x_optim[1])
            iter_counter_list.append(iter_final)
            func_counter_list.append(func_counter)
            grad_counter_list.append(grad_counter)
            
        print(x1_optim_list)            
        x0_optim_arr[ndx, mdx] = x0_optim_list
        x1_optim_arr[ndx, mdx] = x1_optim_list
        iter_counter_arr[ndx, mdx] = iter_counter_list
        func_counter_arr[ndx, mdx] = func_counter_list
        grad_counter_arr[ndx, mdx] = grad_counter_list

[0.007035259520084913, -0.0018436065697543724, -0.00025342502517262727, -2.6423734021689534e-05, -2.6605799076120174e-06]
[0.0071292067021479074, -0.0018430493705112532, -0.0002536411141949295, -2.6463083777637053e-05, -2.6640819218230263e-06]
[0.007137427570889999, -0.0018428675209713082, -0.0002536906004688831, -2.6468900851511898e-05, -2.6645843504551198e-06]
[0.007138407261852228, -0.0018428528828248923, -0.000253692754738691, -2.6469369359404678e-05, -2.664623135890116e-06]
[0.2061163665571661, 0.2061163665571661, 0.2061163665571661, 0.2061163665571661, 0.2061163665571661]


Преобразуем в таблицы.

In [25]:
col_name_x_optim = []
col_name = []

x_optim_tbl_set = np.empty((N_FUNC,), dtype=object)
iter_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
func_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
grad_counter_tbl_set = np.empty((N_FUNC,), dtype=object)

for ndx in range(N_FUNC):
    x_optim_tbl = eps_var.copy()

    for idx, col1, col2 in zip(range(N_LR), x0_optim_arr[ndx, :], x1_optim_arr[ndx, :]):
        x_optim_tbl = np.vstack((x_optim_tbl, col1, col2))
        col_name_x_optim.append('x1 (lr=' + str(lr_classic_grad_descent[idx][0]) + ')')
        col_name_x_optim.append('x2 (lr=' + str(lr_classic_grad_descent[idx][1]) + ')')
    
    x_optim_tbl_set[ndx] = pd.DataFrame(x_optim_tbl.T, columns=['eps'] + col_name_x_optim)

    iter_counter_tbl = eps_var.copy()
    for idx, col in enumerate(iter_counter_arr[ndx, :]):
        iter_counter_tbl = np.vstack((iter_counter_tbl, col))
        col_name.append('x1: lr=' + str(lr_classic_grad_descent[idx][0]) + ', x2: lr=' + str(lr_classic_grad_descent[idx][1]))
    
    iter_counter_tbl_set[ndx] = pd.DataFrame(iter_counter_tbl.T, columns=['eps'] + col_name)
    iter_counter_tbl_set[ndx][col_name] = iter_counter_tbl_set[ndx][col_name].astype(int)

    func_counter_tbl = eps_var.copy()
    for idx, col in enumerate(func_counter_arr[ndx, :]):
        func_counter_tbl = np.vstack((func_counter_tbl, col))
    
    func_counter_tbl_set[ndx] = pd.DataFrame(func_counter_tbl.T, columns=['eps'] + col_name)
    func_counter_tbl_set[ndx][col_name] = func_counter_tbl_set[ndx][col_name].astype(int)

    grad_counter_tbl = eps_var.copy()
    for idx, col in enumerate(grad_counter_arr[ndx, :]):
        grad_counter_tbl = np.vstack((grad_counter_tbl, col))
    
    grad_counter_tbl_set[ndx] = pd.DataFrame(grad_counter_tbl.T, columns=['eps'] + col_name)
    grad_counter_tbl_set[ndx][col_name] = grad_counter_tbl_set[ndx][col_name].astype(int)

In [26]:
x_optim_tbl_set[0]

,eps,x1 (lr=0.001),x2 (lr=0.001),x1 (lr=0.0001),x2 (lr=0.0001),x1 (lr=1e-05),x2 (lr=1e-05),x1 (lr=1e-06),x2 (lr=1e-06),x1 (lr=1e-07),x2 (lr=1e-07)
0,0.10000,0.055558,0.007035,0.055653,0.007129,0.055660,0.007137,0.055661,0.007138,0.283885,0.206116
1,0.01000,0.006594,-0.001844,0.006610,-0.001843,0.006611,-0.001843,0.006611,-0.001843,0.283885,0.206116
2,0.00100,0.000648,-0.000253,0.000649,-0.000254,0.000649,-0.000254,0.000649,-0.000254,0.283885,0.206116
3,0.00010,0.000064,-0.000026,0.000064,-0.000026,0.000064,-0.000026,0.000064,-0.000026,0.283885,0.206116
4,0.00001,0.000006,-0.000003,0.000006,-0.000003,0.000006,-0.000003,0.000006,-0.000003,0.283885,0.206116


In [27]:
iter_counter_tbl_set[0]

,eps,"x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06","x1: lr=1e-07, x2: lr=1e-07"
0,0.10000,1924,19245,192458,1924584,10000000
1,0.01000,3295,32956,329573,3295741,10000000
2,0.00100,4883,48853,488545,4885473,10000000
3,0.00010,6487,64901,649036,6490389,10000000
4,0.00001,8091,80953,809569,8095732,10000000


In [28]:
func_counter_tbl_set[0]

,eps,"x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06","x1: lr=1e-07, x2: lr=1e-07"
0,0.10000,1924,19245,192458,1924584,10000000
1,0.01000,3295,32956,329573,3295741,10000000
2,0.00100,4883,48853,488545,4885473,10000000
3,0.00010,6487,64901,649036,6490389,10000000
4,0.00001,8091,80953,809569,8095732,10000000


In [29]:
grad_counter_tbl_set[0]

,eps,"x1: lr=0.001, x2: lr=0.001","x1: lr=0.0001, x2: lr=0.0001","x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06","x1: lr=1e-07, x2: lr=1e-07"
0,0.10000,1924,19245,192458,1924584,10000000
1,0.01000,3295,32956,329573,3295741,10000000
2,0.00100,4883,48853,488545,4885473,10000000
3,0.00010,6487,64901,649036,6490389,10000000
4,0.00001,8091,80953,809569,8095732,10000000


In [30]:
plot_list = [x_optim_df, iter_counter_df, func_counter_df]
plot_name = ['x-координата минимума функции', 'Количество итераций', 'Количество вычислений функции']
fig_name = ['x_optim', 'iter_number', 'func_calc_number']

tbl_title = [0] * 3

tbl_title[0] = 'Таблица оптимальных значений (' + func_comment + ' функция)'
tbl_title[1] = 'Таблица количества итераций (' + func_comment + ' функция)'
tbl_title[2] = 'Таблица количества вычислений функции (' + func_comment + ' функция)'

for tbl_tlt, plt_lst in zip(tbl_title, plot_list):
    print(tbl_tlt)
    display(plt_lst)

NameError: name 'x_optim_df' is not defined

In [ ]:
#grad_descent_methods = [classic_grad_descent]#, armijo_grad_descent, steep_grad_descent]
#N_METH = len(grad_descent_methods)

